# Building a ChatBot using OpenAI APIs

We will use the updated OpenAI Python SDK (version ≥ 1.0.0), which introduces a more modern API structure using `OpenAI` client instances.

In [3]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Retrieve the API key
api_key = os.getenv("OPENAI_API_KEY")

# Sanity check (should print a masked version)
if api_key:
    print("✅ API key loaded successfully.")
else:
    print("❌ API key not found. Please check your .env file.")

✅ API key loaded successfully.


In [4]:
# Initialize OpenAI client
client = OpenAI(api_key=api_key)

# Single-Turn Prompting with Chat Models

The OpenAI Chat API uses a list of `messages` to simulate a conversation. Each message has a `role`, such as:

- `system`: Sets the behavior or personality of the assistant
- `user`: Represents input from the user
- `assistant`: (optional) Responses from the assistant

In a single-turn prompt, we include just one user message (optionally with a system message), and ask the model for a response.

Let's try a simple example using the `gpt-3.5-turbo` model.


In [5]:
# Simple single-turn chat completion

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": input("     👤 User: ")}
    ],
    temperature=0.0  # Controls randomness; higher = more creative
)

print(f"🤖 Assistant:  {response.choices[0].message.content}")


🤖 Assistant:  India's first Prime Minister was Jawaharlal Nehru. He served as Prime Minister from 1947 to 1964.


# Multi-turn Conversations and Context Memory

In real-world interactions, a chatbot needs to understand the context from previous messages — especially for tasks like:

- Remembering names
- Answering follow-up questions
- Maintaining consistent tone or persona

While OpenAI models don't have persistent memory by default, we can **simulate memory** by including prior messages in the `messages` list.

Let's simulate a short conversation where the assistant remembers the user's name.


In [6]:
# Multi-turn conversation: remembering user's name

conversation = [
    {"role": "system", "content": "You are a friendly assistant."},
    {"role": "user", "content": "Hi, my name is Rajat."},
    {"role": "assistant", "content": "Hi Rajat! Nice to meet you. How can I help you today?"},
    {"role": "user", "content": "Can you remind me what my name is?"}
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=conversation,
    temperature=0.0
)

print(f"🤖 Assistant: {response.choices[0].message.content}")


🤖 Assistant: Of course, your name is Rajat.


# Creating Helper Functions for Chat Completion

To streamline our workflow, we’ll define two helper functions:

1. `get_completion(prompt)` — for single-turn prompts (user + system)
2. `get_completion_from_messages(messages)` — for multi-turn conversations

These functions use the OpenAI client and return the assistant's reply.


In [7]:
# Reusable helper functions for chat completions

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0.0):
    """Single user prompt with optional system role."""
    messages = [{"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    return response.choices[0].message.content


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0.0):
    """Handles full multi-turn message history."""
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    return response.choices[0].message.content

# DocuBot with Document Context (Embedded in System Prompt)

In this version, we simulate a chatbot that can answer questions based on a specific document.

- We inject the document into the system prompt
- DocuBot is instructed to rely only on the provided document
- This simulates how Retrieval-Augmented Generation (RAG) might pass chunked context to the model

**Note:** This example uses a short mock document. In practice, you'd pass in pre-processed chunks from longer files.


In [8]:
# Mock document content (can be replaced with dynamic input later)
document_text = """
This lease agreement specifies that the lessee is responsible for all maintenance \
and repair costs associated with the premises, including plumbing, electrical, and \
structural upkeep. The lessor is only responsible for providing the property in a \
habitable condition at the beginning of the lease term. Termination requires a 60-day \
notice from either party.
"""

In [10]:
# System prompt with embedded document
docubot_system_message = f"""
You are DocuBot, a highly reliable AI assistant specialized in interpreting legal, tax, and healthcare documents.

Your job is to help users understand and summarize the content of the document provided below.

You must only use the information from the document. If a question is outside the document's scope, clearly say so without guessing.

Respond in a concise, formal, and professional tone.

---
DOCUMENT START
{document_text}
DOCUMENT END
---
"""

print(docubot_system_message)


You are DocuBot, a highly reliable AI assistant specialized in interpreting legal, tax, and healthcare documents.

Your job is to help users understand and summarize the content of the document provided below.

You must only use the information from the document. If a question is outside the document's scope, clearly say so without guessing.

Respond in a concise, formal, and professional tone.

---
DOCUMENT START

This lease agreement specifies that the lessee is responsible for all maintenance and repair costs associated with the premises, including plumbing, electrical, and structural upkeep. The lessor is only responsible for providing the property in a habitable condition at the beginning of the lease term. Termination requires a 60-day notice from either party.

DOCUMENT END
---



In [ ]:
# Initialize chat context
context = [{"role": "system", "content": docubot_system_message}]

print("📘 DocuBot is now active with a loaded document. Type 'exit' to stop.\n")

while True:
    user_input = input("👤 User: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Session ended.")
        break

    context.append({"role": "user", "content": user_input})

    response = get_completion_from_messages(context, temperature=0.0)
    print(f"🤖 DocuBot: {response}\n")

    context.append({"role": "assistant", "content": response})


📘 DocuBot is now active with a loaded document. Type 'exit' to stop.

🤖 DocuBot: Hello Himanshu. How can I assist you today?

🤖 DocuBot: According to the lease agreement provided, the lessee is responsible for all maintenance and repair costs associated with the premises, including plumbing, electrical, and structural upkeep.

🤖 DocuBot: I do not have access to personal information beyond what you have shared with me. If there is anything else related to the document or any other topic you need assistance with, feel free to ask.

🤖 DocuBot: If you have any more questions or need further assistance in the future, feel free to ask. Have a great day!

🤖 DocuBot: I apologize for the oversight. Your name is Himanshu. How can I assist you further, Himanshu?



## 🧠 Sample Questions for DocuBot (Based on Lease Agreement)

### ✅ Basic Understanding

* *"Who is responsible for maintenance of the property?"*
* *"What does the lease say about plumbing repairs?"*
* *"Is the lessor responsible for any repairs?"*
* *"Can you summarize the maintenance clause?"*

### ✅ Conditions and Obligations

* *"What does the lease require from the lessee?"*
* *"What condition must the lessor meet at the start of the lease?"*
* *"Does the lease specify how to end the agreement?"*

### ✅ Termination & Exit

* *"What is the notice period for termination?"*
* *"How can either party terminate the lease?"*
* *"Is there any penalty for ending the lease early?"*

### ❌ Out-of-Scope Examples (DocuBot should reject)

* *"What are the tax implications of this lease?"*
* *"Can I sublease the property?"*
* *"Is this lease compliant with New York housing law?"*

DocuBot should politely decline to answer these unless that info is explicitly in the provided document.